## Build the Training container for fastai 

In this notebook, we will build the inference container that can be used for both Batch Transform and End point

In [1]:
#change permission on executable files 
!chmod +x code/*

Build the container locally

In [2]:
!docker build -t fastai .

Sending build context to Docker daemon  22.02kB
Step 1/8 : FROM fastdotai/fastai:2021-02-11
2021-02-11: Pulling from fastdotai/fastai

cc0b8772: Pulling fs layer 
fb62ba5f: Pulling fs layer 
964ece6a: Pulling fs layer 
21e7e7a7: Pulling fs layer 
5cfd2a87: Pulling fs layer 
79fda18c: Pulling fs layer 
d338cee0: Pulling fs layer 
ce662b92: Pulling fs layer 
c9683ef2: Pulling fs layer 
47a309c2: Pulling fs layer 
c2420471: Pulling fs layer 
b700ef54: Pulling fs layer 
209d4da8: Pulling fs layer 
6dd12925: Pulling fs layer 
1bc6d8f6: Pulling fs layer 
854b0ea3: Pulling fs layer 
Digest: sha256:c36b43104474006d8f8cd2a65f740bfd505693c670644c1d2dbedb5a6fb2de8aDownloading  505.1MB/2.471GB
Status: Downloaded newer image for fastdotai/fastai:2021-02-11
 ---> c15a6ed2e7f0
Step 2/8 : LABEL maintainer="Raj Kadiyala"
 ---> Running in 1ecbecb58aa0
Removing intermediate container 1ecbecb58aa0
 ---> 7c3a3c5bba0b
Step 3/8 : WORKDIR /
 ---> Running in 75017cac86be
Removing intermediate container 75017ca

In [3]:
!docker images

REPOSITORY         TAG          IMAGE ID       CREATED                  SIZE
fastai             latest       6ce894314e19   Less than a second ago   7.54GB
fastdotai/fastai   2021-02-11   c15a6ed2e7f0   2 years ago              7.43GB


Authenticate to ECR using boto3 and set names for tags and ecr repository

In [4]:
import boto3
import sagemaker
ecr_namespace = "sagemaker-training-containers/"
prefix = "script-mode-container-fastai"
role = sagemaker.get_execution_role()
ecr_repository_name = ecr_namespace + prefix
account_id = role.split(":")[4]
region = boto3.Session().region_name
tag_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/'+ecr_repository_name+':latest'

tag with ecr repository

In [5]:
!docker tag fastai $tag_name

Create a ECR repo, if it already exists, you will get an error and you can ignore it 

Authenticate into ECR 

In [6]:
!$(aws ecr get-login --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [7]:
!aws ecr create-repository --repository-name $ecr_repository_name

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:973994155064:repository/sagemaker-training-containers/script-mode-container-fastai",
        "registryId": "973994155064",
        "repositoryName": "sagemaker-training-containers/script-mode-container-fastai",
        "repositoryUri": "973994155064.dkr.ecr.us-west-2.amazonaws.com/sagemaker-training-containers/script-mode-container-fastai",
        "createdAt": 1677459054.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


Push the local image to ECR

In [8]:
!docker push $tag_name

The push refers to repository [973994155064.dkr.ecr.us-west-2.amazonaws.com/sagemaker-training-containers/script-mode-container-fastai]

eb300e14: Preparing 
d4bd4caf: Preparing 
aa25ed6c: Preparing 
03119f42: Preparing 
9d9efed5: Preparing 
5dee3f41: Preparing 
e46047de: Preparing 
ea1e71e9: Preparing 
bf18a086: Preparing 
fc49132e: Preparing 
5e116b6d: Preparing 
5da50cc0: Preparing 
722bdc07: Preparing 
b673a1d6: Preparing 
150d2459: Preparing 
6268583e: Preparing 
cc6eae8b: Preparing 
8881187d: Preparing 
5df75b44: Preparing 
6268583e: Pushed   5.531GB/5.5GBGB[17APushing  920.4MB/1.252GBPushing  1.159GB/1.252GBlatest: digest: sha256:9b902f7ff620607be23546084f58bbfa96b5067b5af457c2e0a98dbaa43a36fe size: 4715


In [9]:
tag_name

'973994155064.dkr.ecr.us-west-2.amazonaws.com/sagemaker-training-containers/script-mode-container-fastai:latest'